# 飞书知识库技术原型

## 飞书全局变量

个人版本，尝试 `tenant_access_token`。是可以的。

In [1]:
%%time

app_id = "cli_a6446f9440cad00c"
app_secret = "G0vGE6ZSWhj6rq1ae6UBlhwUViBuTRzY"
space_id="7376486287542894595"

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.58 µs


## 创建飞书客户端

In [2]:
%%time

import lark_oapi as lark

client = lark.Client.builder() \
    .app_id(app_id) \
    .app_secret(app_secret) \
    .build()

CPU times: user 1.12 s, sys: 148 ms, total: 1.27 s
Wall time: 1.27 s


In [3]:
%%time

import requests
import json

# 飞书的 API 端点
url = "https://open.feishu.cn/open-apis/auth/v3/tenant_access_token/internal"

# 请求头
headers = {
    "Content-Type": "application/json; charset=utf-8"
}

# 请求体
payload = {
    "app_id": app_id,
    "app_secret": app_secret
}

# 发起请求获取 tenant_access_token
response = requests.post(url, headers=headers, data=json.dumps(payload))

# 解析响应
if response.status_code == 200:
    data = response.json()
    if data.get("code") == 0:
        tenant_access_token = data.get("tenant_access_token")
        print(f"Tenant Access Token: {tenant_access_token}")
    else:
        print(f"Error: {data.get('msg')}")
else:
    print(f"Failed to get token, status code: {response.status_code}")


Tenant Access Token: t-g1049olHLRX2WAPSDZFZ5PWEFHQMPELRGONFMAZJ
CPU times: user 5.33 ms, sys: 241 µs, total: 5.57 ms
Wall time: 124 ms


In [4]:
%%time

from lark_oapi.api.wiki.v2 import *

request: ListSpaceNodeRequest = ListSpaceNodeRequest.builder() \
        .space_id(space_id) \
        .build()

CPU times: user 59 µs, sys: 0 ns, total: 59 µs
Wall time: 61.3 µs


In [5]:
%%time

req_opt = lark.RequestOption.builder()\
    .tenant_access_token(tenant_access_token)\
    .headers(headers)\
    .build()

response: ListSpaceNodeResponse = client.wiki.v2.space_node.list(request)

CPU times: user 8.06 ms, sys: 529 µs, total: 8.58 ms
Wall time: 593 ms


In [6]:
%%time

if not response.success():
    lark.logger.error(
        f"client.wiki.v2.space_node.list failed, code: {response.code}, msg: {response.msg}, log_id: {response.get_log_id()}")
else:
    lark.logger.info(lark.JSON.marshal(response.data, indent=4))

CPU times: user 137 µs, sys: 19 µs, total: 156 µs
Wall time: 157 µs


In [7]:
response.data.items[0].obj_token

'ILyid3Fc2oCUuWxkS37clBNgnHc'

In [8]:
response.data.items[0].obj_edit_time

'1717472050'